1. Thought
- Choose Tool
2. Action
- Output
3. Observation
- Context

Repeat

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
import os
from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

In [2]:
class Agent:
    def __init__(self, client, system):
        self.client = client
        self.system = system
        self.messages = []
        if self.system is not None:
            self.messages.append( {'role': 'system', 'content': self.system} )

    def __call__(self,message=""):
        if message:
            self.messages.append( {'role': 'user', 'content': message} )
        result = self.execute()
        self.messages.append( {'role': 'assistant', 'content': result} )
        return result
    
    def execute(self):
        completion = client.chat.completions.create(
            messages = self.messages,
            model="llama-3.3-70b-versatile",
        )
        return completion.choices[0].message.content

In [3]:
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_planet_mass:
e.g. get_planet_mass: Earth
returns weight of the planet in kg

Example session:

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of Earth
Action: get_planet_mass: Earth
PAUSE 

You will be called again with this:

Observation: 5.972e24

Thought: I need to multiply this by 2
Action: calculate: 5.972e24 * 2
PAUSE

You will be called again with this: 

Observation: 1,1944×10e25

If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 is 1,1944×10e25.

Now it's your turn:
""".strip()

# Tools
def calculate(operation) -> float:
    return eval(operation)

def get_planet_mass(planet) -> float:
    match planet.lower():
        case "mercury":
            return 3.285e23
        case "venus":
            return 4.867e24
        case "earth":
            return 5.972e24
        case "mars":
            return 6.39e23
        case "jupiter":
            return 1.898e27
        case "saturn":
            return 5.683e26
        case "uranus":
            return 8.681e25
        case "neptune":
            return 1.024e26
        case _:
            return 0.0

In [ ]:
# Loop for the answer
import re

def loop (max_iterations = 3, query = ''):
    agent = Agent(client=client, system=system_prompt)
    tools = ['calculate', 'get_planet_mass']

    next_prompt = query
    i = 0
    while i < max_iterations:
        i += 1
        result = agent(next_prompt)
        print(f'{result}')

        if "PAUSE" in result and "Action" in result:
            action = re.findall( r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE )
            choosen_tool = action[0][0]
            arg = action[0][1]
        
            if choosen_tool in tools:
                result_tool = eval(f"{choosen_tool}('{arg}')")
                next_prompt = f"Observation: {result_tool}"
            else:
                next_prompt = "Observation: Tool not found"
            print(next_prompt)
            continue
        if "Answer" in result:
            break

In [8]:
loop(max_iterations = 5, query="What is the mass of the earth plus the mass of the mercury and all of it times 5?")

Thought: To find the mass of the Earth plus the mass of Mercury and then multiply the sum by 5, I first need to find the mass of the Earth and the mass of Mercury. 

Action: get_planet_mass: Earth
PAUSE
Observation: 5.972e+24
Thought: I now have the mass of the Earth, which is 5.972e+24 kg. Next, I need to find the mass of Mercury.

Action: get_planet_mass: Mercury
PAUSE
Observation: 3.285e+23
Thought: I now have the masses of both the Earth (5.972e+24 kg) and Mercury (3.285e+23 kg). The next step is to add these two masses together and then multiply the sum by 5.

Action: calculate: (5.972e+24 + 3.285e+23) * 5
PAUSE
Observation: 3.1502500000000004e+25
Thought: I have now calculated the sum of the masses of the Earth and Mercury, and then multiplied this sum by 5, resulting in 3.1502500000000004e+25 kg. This is the final step in solving the problem.

Answer: The mass of the Earth plus the mass of Mercury and all of it times 5 is 3.15025e+25 kg.


## Let's just print the answer

In [19]:
def output (max_iterations = 3, query = ''):
    agent = Agent(client=client, system=system_prompt, )
    tools = ['calculate', 'get_planet_mass']

    next_prompt = query
    i = 0
    while i < max_iterations:
        i += 1
        result = agent(next_prompt)
        # print(f'{result}')

        if "PAUSE" in result and "Action" in result:
            action = re.findall( r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE )
            choosen_tool = action[0][0]
            arg = action[0][1]
        
            if choosen_tool in tools:
                result_tool = eval(f"{choosen_tool}('{arg}')")
                # next_prompt = f"Observation: {result_tool}"
            else:
                next_prompt = "Observation: Tool not found"
            
            # print(next_prompt)
            continue
        if "Answer" in result:
            break
    return result.split("Answer: ", 1)[-1]

answer = output(max_iterations = 5, query="What is the mass of the earth plus the mass of the mercury and all of it times 5?")
print(answer)

The mass of the earth plus the mass of Mercury and all of it times 5 is 3.1011e25.


In [ ]:
#instead of eval, use this: ChatGPT due to security concerns.
"""
import ast
import operator

def safe_calculate(expression: str) -> float:
    allowed_operators = {
        ast.Add: operator.add,
        ast.Sub: operator.sub,
        ast.Mult: operator.mul,
        ast.Div: operator.truediv,
        ast.Pow: operator.pow,
        ast.Mod: operator.mod,
    }

    def evaluate(node):
        if isinstance(node, ast.BinOp):
            left = evaluate(node.left)
            right = evaluate(node.right)
            if type(node.op) in allowed_operators:
                return allowed_operators[type(node.op)](left, right)
            else:
                raise ValueError("Unsupported operation")
        elif isinstance(node, ast.Num):
            return node.n
        else:
            raise ValueError("Invalid expression")

    try:
        parsed_expr = ast.parse(expression, mode='eval')
        return evaluate(parsed_expr.body)
    except Exception:
        raise ValueError("Invalid input")

# Example usage:
print(safe_calculate("3 + 5 * 2"))  # Output: 13

"""